In [25]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Reading the Dataset
dataset = pd.read_csv('CKD.csv')
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3,0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2,0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1,0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1,0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0,0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0,0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0,2,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3,0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0,0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [24]:
#Checking dataset for balanced or imbalanced:
dataset["classification_yes"].value_counts()
#dataset.columns

classification_yes
1    249
0    150
Name: count, dtype: int64

In [4]:
#split into independent and dependent variable:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset['classification_yes']

In [5]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [6]:
from sklearn.linear_model import LogisticRegression
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag',  'saga'],
              'multi_class' : ['auto', 'ovr', 'multinomial'],
             'penalty':['l1','l2','elasticnet', 'None']} 

grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'multi_class': ['auto', 'ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet', 'None'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='f1_weighted', verbose=3)

In [7]:
# print best parameter after tuning 
print(grid.best_params_) 

{'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'newton-cg'}


In [8]:
re=grid.cv_results_
#print(re)

In [9]:
grid_predictions = grid.predict(X_test)  
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [10]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [11]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'newton-cg'}: 0.9924667654735397


In [12]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[50  1]
 [ 0 82]]


In [13]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99        51
           1       0.99      1.00      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [14]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

1.0

In [15]:
table=pd.DataFrame.from_dict(re)

In [16]:
table
#table.loc[:,"params"]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_multi_class,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004690,0.000602,0.000000,0.000000,auto,l1,newton-cg,"{'multi_class': 'auto', 'penalty': 'l1', 'solv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,20
1,0.002895,0.000194,0.000000,0.000000,auto,l1,lbfgs,"{'multi_class': 'auto', 'penalty': 'l1', 'solv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,20
2,0.013970,0.001418,0.008253,0.001158,auto,l1,liblinear,"{'multi_class': 'auto', 'penalty': 'l1', 'solv...",1.000000,0.943041,0.981217,0.943651,1.000000,0.973582,0.025623,4
3,0.003590,0.001195,0.000000,0.000000,auto,l1,sag,"{'multi_class': 'auto', 'penalty': 'l1', 'solv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,20
4,0.035856,0.002426,0.008991,0.002186,auto,l1,saga,"{'multi_class': 'auto', 'penalty': 'l1', 'solv...",0.486532,0.501410,0.477841,0.477841,0.477841,0.484293,0.009196,11
5,0.151602,0.012886,0.006898,0.000483,auto,l2,newton-cg,"{'multi_class': 'auto', 'penalty': 'l2', 'solv...",0.981569,0.981014,0.962573,0.943093,0.981031,0.969856,0.015204,6
6,0.041065,0.002059,0.007292,0.000927,auto,l2,lbfgs,"{'multi_class': 'auto', 'penalty': 'l2', 'solv...",0.869648,0.905069,0.906705,0.906085,0.885478,0.894597,0.014791,9
7,0.010508,0.002046,0.007978,0.001784,auto,l2,liblinear,"{'multi_class': 'auto', 'penalty': 'l2', 'solv...",0.981569,0.981014,0.962573,0.962264,1.000000,0.977484,0.014072,2
8,0.023059,0.003067,0.007386,0.001456,auto,l2,sag,"{'multi_class': 'auto', 'penalty': 'l2', 'solv...",0.486532,0.501410,0.477841,0.477841,0.477841,0.484293,0.009196,11
9,0.023732,0.003186,0.007672,0.001254,auto,l2,saga,"{'multi_class': 'auto', 'penalty': 'l2', 'solv...",0.486532,0.501410,0.477841,0.477841,0.477841,0.484293,0.009196,11


In [17]:
# Deployement Phase:
import pickle
filename="finalized_model_CKD.sav"
pickle.dump(grid,open(filename,'wb'))

In [18]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [19]:
loaded_model=pickle.load(open("finalized_model_CKD.sav",'rb'))
result=loaded_model.predict([[22,20,3,2,250,300,8,280,5,7,25,8000,5,1,0,0,0,1,0,1,0,0,1,1,0,1,1]])
result

array([1])

In [22]:
age = float(input("Age:")) #age = 1 to 100
bp = float(input("Blood pressure:")) #bp = 10 to 150
al = float(input("Alostatic load:")) #al = 0 to 4
su = float(input("Substance Use:")) #su = 0 to 4
bgr = float(input("Blood Glucose Regulator:")) #bg = 10 to 500
bu = float(input("Buruli ulcer:")) #bu = 10 to 500
sc = float(input("Sickle Cell:")) #sc = 1 to 10
sod = float(input("Sphincter of Oddi Dysfunction:")) #sod = 10 to 500
pot = float(input("Postural Orthostatic Tachycardia:")) #pod = 1 to 10 
hrmo = float(input("Highly Resistant Microorganism:")) #hrmo = 1 to 15
pcv = float(input("Packed Cell Volume:")) #pcv = 1 to 100    
wc = float(input("Waist Circumference:")) #wc = 100 to 10000 
rc = float(input("radical cystectomy:")) #rc = 1 to 10
sg_b = float(input("Specific Gravity b:")) #sg (a = 0, b = 1, c = 2, d = 3)
sg_c = float(input("Specific Gravity c:")) 
sg_d = float(input("Specific Gravity d:")) 
sg_e = float(input("Specific Gravity e:"))
rbc_normal = float(input("Red Blood Cell:")) #rbc (yes= 1, no= 0)
pc_normal = float(input("Palliative Care:")) #pc (normal = 1, abnormal = 0)
pcc_present = float(input("Patient Centered Care:")) #pcc (present = 1, not present= 0)
ba_present = float(input("Bronchial Asthma:")) #ba (present = 1, not present= 0)
htn_yes = float(input("Hypertension:")) #htn (yes=1, no=0)
dm_yes = float(input("Diabetes Mellitus:")) #dm (yes= 1, no= 0)
cad_yes = float(input("Coronary artery disease :")) #cad (yes= 1, no= 0)
appet_yes = float(input("Amiodarone-induced Pulmonary Toxicity:")) #appet (yes= 1, no= 0)
pe_yes = float(input("Pulmonary Embolism:")) #pe (yes= 1, no= 0)
ane_yes = float(input("Anemia of Chronic:")) #ane (yes= 1, no= 0)

Age: 22
Blood pressure: 20
Alostatic load: 3
Substance Use: 2
Blood Glucose Regulator: 250
Buruli ulcer: 300
Sickle Cell: 8
Sphincter of Oddi Dysfunction: 280
Postural Orthostatic Tachycardia: 5
Highly Resistant Microorganism: 7
Packed Cell Volume: 25
Waist Circumference: 8000
radical cystectomy: 5
Specific Gravity b: 1
Specific Gravity c: 0
Specific Gravity d: 0
Specific Gravity e: 0
Red Blood Cell: 1
Palliative Care: 0
Patient Centered Care: 1
Bronchial Asthma: 0
Hypertension: 0
Diabetes Mellitus: 1
Coronary artery disease : 1
Amiodarone-induced Pulmonary Toxicity: 0
Pulmonary Embolism: 1
Anemia of Chronic: 1


In [23]:
Future_Prediction=grid.predict([[age, bp, al, su, bgr, bu, sc, sod, pot, hrmo, pcv,
       wc, rc, sg_b, sg_c, sg_d, sg_e, rbc_normal, pc_normal,
       pcc_present, ba_present, htn_yes, dm_yes, cad_yes,
       appet_yes, pe_yes, ane_yes]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
